# Text classification on movie reviews Dataset from stanford with Bert

In [ ]:
fig = dict({
    "data": [{"type": "bar",
              "x": [1, 2, 3],
              "y": [1, 3, 2]}],
    "layout": {"title": {"text": "A Figure Specified By Python Dictionary"}}
})

# To display the figure defined by this dict, use the low-level plotly.io.show function
import plotly.io as pio

pio.show(fig)

/content/drive/MyDrive/transformers_for_nlp


/content/drive/MyDrive/transformers_for_nlp


In [ ]:
!pip install transformers


     |████████████████████████████████| 2.1MB 5.3MB/s 
     |████████████████████████████████| 3.3MB 16.7MB/s 
     |████████████████████████████████| 870kB 45.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=aebcffdcc9b096bd0f9bce25864c284bb01f739ed1626927a57d7d7ce04aef06
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
# Downloading the dataset.
#!wget -q -nc http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


In [ ]:
# Unziping the dataset.
#!tar -zxf /content/drive/MyDrive/transformers_for_nlp/aclImdb_v1.tar.gz

In [ ]:
!pip install -q git+https://github.com/gmihaila/ml_things.git


     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 10.3MB 6.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
import os
import io
from tqdm.notebook import tqdm
from torch.utils.data import Dataset , DataLoader 
from ml_things import plot_dict , fix_text , plot_confusion_matrix
from sklearn.metrics import classification_report , accuracy_score
from transformers import (AutoConfig ,AutoModelForSequenceClassification ,AutoTokenizer,AdamW,
                          get_linear_schedule_with_warmup , set_seed)


/usr/local/lib/python3.7/dist-packages/packaging/version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,


In [ ]:
import torch
from torch import nn 
import torch.nn.functional as F

In [ ]:
set_seed(26)
epochs = 4
batches =  32 # as max seq length is small but for max _Seq_len =512 batch_Size should be very small (due togpu issues)
max_seq_length = 50
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cpu' :
    print("Using cpu ,This will b slow")
model_name = 'bert-base-cased'

labels_d = { "pos" : 1 , "neg" : 0}
n_labels = len(labels)



In [ ]:
class MRDataSet(Dataset):
    r"""
    This class can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.
 
  Arguments:
 
    path (:obj:`str`):
        Path to the data partition.
     
    tokenizer (:obj:`transformers.tokenization_?`):
        Transformer type tokenizer used to process raw text into numbers.
 
    labels (:obj:`dict`):
        Dictionary to encode any labels names into numbers. Keys map to 
        labels names and Values map to number associated to those labels.
 
    max_sequence_len (:obj:`int`, `optional`)
        Value to indicate the maximum desired sequence to truncate or pad text
        sequences. If no value is passed it will used maximum sequence size
        supported by the tokenizer and model.
     """
    def __init__(self , path = "/content/drive/MyDrive/transformers_for_nlp/aclImdb/train"  ,\
                tokenizer = None , \
                labels_d =labels_d, max_seq_len = max_seq_length):
        super(MRDataSet , self).__init__()

        if not os.path.isdir(path):
             print("Path to directory does not exist.")

        texts = []
        labels = []

        #the labels are defined by folders with data we loop through each label.

        for label , label_id in tqdm(labels_d.items()):
             text_path  = os.path.join(path , label) #path/pos | path/neg

             all_file_names = os.listdir(text_path)[:1000]
             for file_name in tqdm(all_file_names):
                 file_path = os.path.join(text_path , file_name)

                 text = io.open(file_path , mode = 'r' , encoding = 'utf-8').read()
                 text = fix_text(text) #solves any encoding issues
                 texts.append(text)
                 labels.append(label_id)
        self.n = len(labels)
        #using tokenizer on text ,returns a dictionary with tokenized integer ids under input_ids key
        self.tokenized_inputs = tokenizer(texts ,add_special_tokens = True , truncation = True, padding = True ,
                                          return_tensors = 'pt' , max_length = max_seq_len)
        self.seq_len = self.tokenized_inputs['input_ids'].shape[-1] #length of sequence

        self.tokenized_inputs.update({"labels" : torch.tensor(labels)})

    def __len__(self) :
            return self.n

    def __getitem__(self , index):
            r"""
            Arguments:
 
            index (:obj:`int`):
                Index position to pick an example to return.
        
            Returns:
            :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
            It holddes the statement `model(**Returned Dictionary)`.

            """
            return {key : self.tokenized_inputs[key][items] for key in self.tokenized_inputs.keys()}







In [ ]:
d = Dataset("/content/drive/MyDrive/transformers_for_nlp/aclImdb/train" , AutoTokenizer , labels , max_seq_len)

TypeError: ignored

In [ ]:
def train(dataloader , optimizer_ , scheduler_ , device_):
    r"""

    Performs a single training pass of the " model " which is a global variable loaded on device_

    Arguments:

    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
        Parsed data into batches of tensors.

    optimizer_ (:obj:`transformers.optimization.AdamW`):
        Optimizer used for training.

    scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
        PyTorch scheduler.

    device_ (:obj:`torch.device`):
        Device used to load tensors before feeding to model.

    Returns:

    :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss].
    """

    global model

    preds , true_labels = [] , [] 
    total_loss = 0
    model.train()

    for batch in tqdm(dataloader , total = len(dataloader)) :
        #Adding to original labels - to be used later for evaluation
        true_labels += batch["labels"].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        model.zero_grad()
        #This will return the loss (rather than the model output) because we have provided the `labels`.
        # model always returns a tuple 
        out = model(**batch)

        loss , logits = out[:2]

        total_loss += loss.item()
        loss.backward()
        #gradient clipping `
        torch.nn.utils.clip_grad_norm_(model.parameters() , 1.0)

        optimizer_.step()
        schedular_.step

        logits = logits.detach().cpu().numpy()
        # Convertion of these logits to list of predicted labels values.
        preds += logits.argmax(axis = -1).flatten().tolist()

    average_epoch_loss = total_loss / len(dataloader)

    return true_labels,preds,average_epoch_loss



 

In [ ]:
def valiadation(dataloader , device_):
    """
    function that performs validation on validation data
     Arguments:
 
    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.
 
    device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.
 
     Returns:
     
    :True Labels, Predictions,  Average Loss

    """

    global model 
    preds , true_labels= [] , []
    total_loss = 0 

    model.eval()

    for batch in tqdm(dataloader , total = len(dataloader)) :
        #Adding to original labels - to be used later for evaluation
        true_labels += batch["labels"].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        with torch.no_grad():
            out = model(**batch)
            loss , logits = out[:2]
            logits = logits.detach().cpu().numpy()
            total_loss += loss

            preds += logits.argmax(axis = -1).flatten().tolist()

    average_validation_loss = total_loss / len(dataloader)

    return true_labels , preds , average_validation_loss


In [ ]:
model_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_name , num_labels = n_labels)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name , config=model_config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
model.to(device)
Tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)



In [ ]:

training_dataset = MRDataSet( tokenizer = Tokenizer )


traning_dataloader = DataLoader(training_dataset , batch_size = batch_size , shuffle =True)


In [ ]:
validation_dataset = MRDataSet( tokenizer = Tokenizer)
validation_dataloader = DataLoader(validation_dataset)

In [ ]:
optimizer = AdamW(model.parameters()  ,lr = 2e-5  ,eps = 1e-8)
# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs
 
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
 
# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}
 
# Loop through each epoch.
print('Epoch')
for epoch in tqdm(range(epochs)):
  print()
  print('Training on batches...')
  # Perform one full pass over the training set.
  train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)
 
  # Get prediction form model on validation data. 
  print('Validation on batches...')
  valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
  val_acc = accuracy_score(valid_labels, valid_predict)
 
  # Print loss and accuracy values to see how training evolves.
  print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
  print()
 
  # Store the loss value for plotting the learning curve.
  all_loss['train_loss'].append(train_loss)
  all_loss['val_loss'].append(val_loss)
  all_acc['train_acc'].append(train_acc)
  all_acc['val_acc'].append(val_acc)
 
# Plot loss curves.
plot_dict(all_loss, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])
 
# Plot accuracy curves.
plot_dict(all_acc, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])